In [1]:
import tensorflow as tf
import os
import numpy as np
import random

random.seed(42)

tfrecords_dir='data/AWID3_tfrecords'

In [ ]:
tfrecords = [os.path.join(tfrecords_dir, file) for file in os.listdir(tfrecords_dir) if file.endswith('.tfrecord')]
epochs = 30

random.shuffle(tfrecords)
test_ratio = 0.8
train_size = int(len(tfrecords) * test_ratio)
train_data = tfrecords[:train_size]
test_data = tfrecords[train_size:]


print("Training set:")
train_names = [t.split('/')[-1].split('.')[0] for t in train_data]
for t in train_names:
    print(t)

print("\nTest set:")
test_names = [t.split('/')[-1].split('.')[0] for t in test_data]
for t in test_names:
    print(t)

Training set:
AWID3_tfrecords\Deauth_29
AWID3_tfrecords\Evil_Twin_52
AWID3_tfrecords\RogueAP_38
AWID3_tfrecords\(Re)Assoc_31
AWID3_tfrecords\RogueAP_33
AWID3_tfrecords\Deauth_22
AWID3_tfrecords\Evil_Twin_63
AWID3_tfrecords\Kr00k_40
AWID3_tfrecords\Evil_Twin_73
AWID3_tfrecords\Kr00k_48
AWID3_tfrecords\(Re)Assoc_23
AWID3_tfrecords\RogueAP_36
AWID3_tfrecords\Kr00k_49
AWID3_tfrecords\Kr00k_44
AWID3_tfrecords\Evil_Twin_55
AWID3_tfrecords\Kr00k_37
AWID3_tfrecords\Kr00k_53
AWID3_tfrecords\Disas_32
AWID3_tfrecords\Deauth_24
AWID3_tfrecords\Evil_Twin_39
AWID3_tfrecords\Deauth_27
AWID3_tfrecords\Kr00k_52
AWID3_tfrecords\(Re)Assoc_29
AWID3_tfrecords\Disas_37
AWID3_tfrecords\Kr00k_51
AWID3_tfrecords\Evil_Twin_40
AWID3_tfrecords\Disas_40
AWID3_tfrecords\Disas_31
AWID3_tfrecords\Kr00k_55
AWID3_tfrecords\Evil_Twin_38
AWID3_tfrecords\Disas_33
AWID3_tfrecords\Kr00k_31
AWID3_tfrecords\Evil_Twin_61
AWID3_tfrecords\Evil_Twin_50
AWID3_tfrecords\Evil_Twin_44
AWID3_tfrecords\Kr00k_34
AWID3_tfrecords\Krack_25

In [ ]:
sequence_length = 64
sequence_shift = 56
batch_size = 200

import data_utils
train_ds = data_utils.create_sequential_dataset(train_data, seq_length=sequence_length, seq_shift=sequence_shift, batch_size=batch_size)
test_ds = data_utils.create_sequential_dataset(test_data, seq_length=sequence_length, seq_shift=sequence_shift, batch_size=batch_size)

In [ ]:
import cnn_lstm

cnn_lstm_model = cnn_lstm.CNN_LSTM_model()

cnn_lstm_model.fit(
        train_ds,
        epochs=epochs,
        callbacks = [cnn_lstm.checkpoint_callback],
    )

Epoch 1/3
     80/Unknown 479s 429ms/step - accuracy: 0.7624 - loss: nan - precision: 0.1406 - recall: 0.0708

In [ ]:
cnn_lstm_model.summary()

cnn_lstm_model.evaluate(test_ds)

In [ ]:
import lstm

lstm_model = lstm.LSTM_model()


lstm_model.fit(
        train_ds,
        epochs=epochs,
        callbacks = [lstm.checkpoint_callback],
    )


In [ ]:
lstm_model.summary()

lstm_model.evaluate(test_ds)

In [ ]:
import gru


gru_model = gru.GRU_model()

gru_model.fit(
        train_ds,
        epochs=epochs,
        callbacks = [gru.checkpoint_callback],
    )


In [ ]:
gru_model.summary()

gru_model.evaluate(test_ds)